In [ ]:
# %pip install ipympl

In [ ]:
# %matplotlib widget
import numpy as np
import caiman as cm
import matplotlib.pyplot as plt
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf
from caiman.source_extraction.cnmf.params import CNMFParams
from caiman.utils.visualization import view_quilt
import pandas as pd
import sciebo
import holoviews as hv
import bokeh.plotting as bpl
bpl.output_notebook()
hv.notebook_extension('bokeh')

In [ ]:
sciebo.download_file_from_sciebo('https://uni-bonn.sciebo.de/s/RR7qj7tklW1rX25', 'data', 'Sue_2x_3000_40_-46.tif')

# 2-Photon Source Extraction

## Preparing data for Source Extraction

In [ ]:
fname = "data/Sue_2x_3000_40_-46.tif"
movie_orig = cm.load(fname)

In [ ]:
# data specific parameters
params = CNMFParams()
data_dependent_params = {
    'fr': 30,
    'decay_time': 0.4,
    'dxy': (2., 2.)
}
params.data.update(data_dependent_params)

Set motion parameters to the following values

'strides' to (48, 48),
'overlaps' to (24, 24),
'max_shifts' to (6, 6),
'max_deviation_rigid' to 3,
'pw_rigid' to True

In [ ]:
params.motion.update(motion_params)

In [ ]:
# motion specific parameters
motion_params = {
    'strides': (48, 48),
    'overlaps': (24, 24),
    'max_shifts': (6, 6),
    'max_deviation_rigid': 3,
    'pw_rigid': True
}
params.motion.update(motion_params)

Perform motion correction

In [ ]:
mc = MotionCorrect(fname, **params.motion)
mc.motion_correct(save_movie=True)
mc.fname_tot_els

Let's see how to save the motion corrected frames as a C order memory map

In [ ]:
# saving a C order memory map
mc_fname = cm.save_memmap(
    mc.fname_tot_els,
    base_name='memmap_',
    order='C'
)

Now we have to read the memory map as an image

In [ ]:
Yr, dims, num_frames = cm.load_memmap(mc_fname)
images = np.reshape(Yr.T, [num_frames] + list(dims), order='F')
images.shape

## Setting Parameters

1. Is the patch width at least three times the width of a neuron?
2. Do individual neurons fit in the overlap region? 
3. Is gSig about half the width of neuron? 
4. K: how many neurons are in each patch? Upper bound.

**Example** Plot the patches with rf=15 and stride=10 and answer the first two questions

In [ ]:
patch_params = {
    "rf": 3,
    "stride": 1
}
params.patch.update(patch_params)
cnmf_model = cnmf.CNMF(n_processes=1, params=params)

cnmf_patch_width = cnmf_model.params.patch['rf']*2 + 1
cnmf_patch_overlap = cnmf_model.params.patch['stride'] + 1
cnmf_patch_stride = cnmf_patch_width - cnmf_patch_overlap

correlation_image = cm.local_correlations(images, swap_dim=False)

patch_ax = view_quilt(correlation_image, 
                      cnmf_patch_stride, 
                      cnmf_patch_overlap, 
);
patch_ax.set_title(f'CNMF Patches Width {cnmf_patch_width}, Overlap {cnmf_patch_overlap}');

Plot the patches with rf=5 and stride=3 and answer the first two questions

Plot the patches with rf=15 and stride=3 and answer the first two questions

Plot the patches with rf=15 and stride=10 and answer the first two questions

Look at the densest region of the median image. What do you think is a good initial estimate of K and gSig. K is the number of neurons in the densest part of the image (It is better to overestimate than underestimate because if the results are wrong, you can refit the whole model by lowering k.) gSig should be roughly half the length and width of a neuron in pixels. 

Set appropriate value to gSig and K in the below cell

In [ ]:
gsig_K = {
    'gSig': #set value,
    'K': #set value
    }

cnmf_model.params.change_params(gsig_K)


## CNMF Model

In [ ]:
cnmf_fit = cnmf_model.fit(images)
cnmf_fit.estimates.plot_contours_nb(img=correlation_image)

In [ ]:
cnmf_refit = cnmf_fit.refit(images)
cnmf_refit.estimates.plot_contours_nb(img=correlation_image);

## Exploring the Estimates Class

**Example** How many components were identified by `cnmf_fit`

In [ ]:
cnmf_fit.estimates.C.shape

How many components were identified by `cnmf_refit`. What is the difference?

How many background components in `cnmf_fit`?

In [ ]:
cnmf_fit.estimates.b.shape

How many background components in `cnmf_refit`?

In [ ]:
cnmf_refit.estimates.b.shape

**Example** Plot the denoised calcium traces of first component from refit

In [ ]:
plt.plot(cnmf_refit.estimates.C[0]);

Plot the denoised calcium traces of second component from refit

Plot the spike count estimate of second component from refit

Plot the spike count estimate of the last component from refit

Let's see how to store the result as a csv file

In [ ]:
frame_rate = cnmf_refit.params.data['fr']
frame_pd = 1/frame_rate
frame_times = np.linspace(0, num_frames*frame_pd, num_frames);

data_to_save = np.vstack((frame_times, cnmf_refit.estimates.S)).T 
save_df = pd.DataFrame(data_to_save)
save_df.rename(columns={0:'time'}, inplace=True)
# save_df.to_csv('2p_spike_counts.csv')

## Other exercises!


1. Go back to CNMF and set K, gSig parameters to very small or very large values. Set rf to None. What does it do?
2. Set merge_thr to very small or very high values: merge_thr is the threshold above which any two overlapping components will be treated as one. What does this do to the extracted components? 
3. The current notebook covers only part of what can be done with caiman. Follow the below link [CNMF source extraction tutorial](https://github.com/flatironinstitute/CaImAn/blob/main/demos/notebooks/demo_pipeline.ipynb) to extract '$\Delta F/F$' values and view denoised movie.

## References

1. `data_endoscope.tif` 1-photon microendoscopic data from mouse dorsal striatum [Reference](https://elifesciences.org/articles/28728#s3).
2. [CNMF source extraction tutorial](https://github.com/flatironinstitute/CaImAn/blob/main/demos/notebooks/demo_pipeline.ipynb)